# KNIT7
# Generate ELK Snapshot on Meas Node
### This notebook creates an ELK snapshot on the measurement node and helps you copy it to a backup destinaton

### Prerequisites
- Slice needs to be created: [Prepare a Slice](./knit7_prepare_a_slice_via_images.ipynb)
- Slice needs to be instrumentized: [Instrumentize a Slice](./knit7_instrumentize_a_slice.ipynb)
---

## Setup

#### Set Slice Name

In [ ]:
slice_name = "MyMonitoredSlice"

#### Create MFLib and ELK export service objects

In [ ]:
from mflib.data_transfer import ElkExporter
elk_tools = ElkExporter(slice_name=slice_name)

#### Display the current Kibana Tunnel

In [ ]:
# The ELK service was created by the elk_tools.instrumentize call.
# Get access info for Kibana by using the mflib.info call to the elk service. (elk_tools is an mflib object).
# Create a dictionary to pass to the service.
data = {}
# Set the info you want to get.
data["get"] = ["nginx_id", "nginx_password"]
# Call info using service name and data dictionary.
info_results = elk_tools.info("elk", data)
print(info_results)

if info_results["success"]:
    print(f"user: {info_results['nginx_id']} \npass: {info_results['nginx_password']}")
print(elk_tools.kibana_tunnel)

print(f"Browse to http://localhost:{elk_tools.kibana_tunnel_local_port}/")

---
## Create an ELK Snapshot

#### Choose name for the repository and snapshots you will create

In [ ]:
repository_name= "backup_repository"
snapshot_name =  "snapshot_test"

#### Create a snapshot repository

In [ ]:
elk_tools.create_repository(repository_name=repository_name)

#### View indices

In [ ]:
elk_tools.view_indices()

#### Create snapshot

In [ ]:
elk_tools.create_snapshot(repository_name=repository_name, snapshot_name=snapshot_name)

---
## Export snapshot to Measurement node as tar file

In [ ]:
elk_tools.export_snapshot_tar(snapshot_name=snapshot_name)

---
## Copy snapshot off Measurement node

### Option 1: Copy to a local device with SCP

##### Prerequisite
- For this step, you need your local ssh environment configured ([Configure Environment](./configure.ipynb))

##### Example SCP (Secure Copy) command to download snapshot file from VM

scp -F ~/.ssh/fabric/ssh_config -i ~/.ssh/fabric/slice_key ubuntu@\[2620:0:c80:1001:f816:3eff:fe81:80cc]:/home/mfuser/services/elk/files/snapshots/snapshot.tar .

##### Fill in the variables below then run to generate your custom SCP command

In [ ]:
# Path of directory to place your snapshot file
directory_path = "/your/directory/"

print(elk_tools.generate_scp_download_command(snapshot_name, directory_path))

##### More info
Fabric guide for transferring data directly from VM to local machine:
https://learn.fabric-testbed.net/knowledge-base/transferring-data-to-and-from-your-vms/

### Option 2: Copy to Jupyter Hub server

##### **CAUTION** The JH servers have very limited storage. If you are doing this method, make sure you do not download large or multiple snapshot files.
##### Fill in the variables below then run to download file to JH server

In [ ]:
# Snapshot will be available in the data_transfer_service/snapshots directory
elk_tools.node.download_file(f"./snapshots/{snapshot_name}.tar", f"/home/mfuser/services/elk/files/snapshots/{snapshot_name}.tar")

---

# Export complete

For import instructions, refer to [Elk Import](./knit7_elk_import.ipynb)